<a href="https://colab.research.google.com/github/audreyemmely/comp-evolucionaria/blob/main/tsp_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **O problema**

Nesse notebook, usaremos um AG para encontrar uma solução para o problema do caixeiro viajante (travelling salesman problem, TSP). O TSP é descrito da seguinte forma:

*“Dada uma lista de cidades e as distâncias entre cada par de cidades, qual é o trajeto mais curto possível que visita cada cidade e retorna à cidade de origem?”*

![image.png](https://miro.medium.com/max/512/1*DwB6K0KKNug7-y59Uowhsg.png)

Com isso, há duas regras importantes para manter em mente:
1.   Cada cidade precisa ser visitada exatamente uma vez
2.   Devemos retornar à cidade de partida, então nossa distância total precisa ser calculada de acordo

# **A abordagem**



*   Gene: uma cidade (representada como coordenadas (x, y))
*   Indivíduo (também conhecido como "cromossomo"): uma única rota que satisfaça as condições acima
*   População: uma coleção de rotas possíveis (ou seja, coleção de indivíduos)
*   Pais: duas rotas que são combinadas para criar uma nova rota